# BDH Legal Violation Classifier — Notebook B (Fixed)
### Kaggle Dual-T4 · Top-8 ECHR Articles · All Fixes Applied

| Fix | What changed | Expected gain |
|-----|-------------|---------------|
| **Bidirectional attention** | Removed `tril()` mask — encoder now sees full context | +5–10 F1 |
| **Hierarchical chunking** | Split long docs into chunks, pool chunk embeddings | +3–8 F1 |
| **`pos_weight` restored** | BCE re-weighted per label for class imbalance | +2–4 F1 |
| **Per-label threshold** | Optimal threshold found per article on val set | +1–3 F1 |
| **Dual-GPU via DataParallel** | Both Kaggle T4s used automatically | ~2× throughput |
| **Tokenizer parallelism fix** | `TOKENIZERS_PARALLELISM=false` before any fork | No more warning |


In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install scikit-learn tqdm matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 19.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 51.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 68.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 58.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 124.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [matplotlib]0 [matplotlib]n]
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install transformers>=4.40.0

Note: you may need to restart the kernel to use updated packages.


In [5]:
#  1. ENVIRONMENT SETUP 
import os

# Fix tokenizer parallelism warning  must be set BEFORE importing tokenizers
os.environ["TOKENIZERS_PARALLELISM"] = "false"


print(f"TOKENIZERS_PARALLELISM = {os.environ['TOKENIZERS_PARALLELISM']}")


TOKENIZERS_PARALLELISM = false


In [6]:
#  2. IMPORTS & SEED 
import json, zipfile, shutil, random, math, time, warnings
from pathlib import Path
from collections import Counter
from dataclasses import dataclass
from typing import Optional, List

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler, autocast
from transformers import AutoTokenizer
from sklearn.metrics import f1_score, precision_score, recall_score
import matplotlib
matplotlib.use("Agg")          # no display needed on Kaggle
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

#  Reproducibility 
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

#  Dual-GPU setup 
n_gpus = torch.cuda.device_count()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"PyTorch : {torch.__version__}")
print(f"GPUs    : {n_gpus}")
for i in range(n_gpus):
    props = torch.cuda.get_device_properties(i)
    print(f"  GPU {i}: {props.name}  ({props.total_memory/1e9:.1f} GB)")

VRAM = torch.cuda.get_device_properties(0).total_memory / 1e9 if n_gpus > 0 else 0
USE_DP = n_gpus > 1          # DataParallel flag  True on Kaggle dual-T4
print(f"DataParallel : {USE_DP}  (will use {n_gpus} GPU{'s' if n_gpus>1 else ''})")


PyTorch : 2.10.0+cu126
GPUs    : 1
  GPU 0: NVIDIA A100-SXM4-40GB  (42.3 GB)
DataParallel : False  (will use 1 GPU)


## Section 1 — Data

In [14]:
import requests
from pathlib import Path
import zipfile
import os

DATA_ROOT = Path("/workspace/ecthr_data")
DATA_ROOT.mkdir(parents=True, exist_ok=True)

print("Downloading ECHR dataset from Kaggle API...")

# Kaggle API endpoint for dataset download
dataset_name = "mathurinache/ecthrnaacl2021"
api_key = os.environ.get("KAGGLE_API_KEY", "")
username = os.environ.get("KAGGLE_USERNAME", "")

# Create auth tuple
auth = (username, api_key)

# Kaggle API URL
url = f"https://www.kaggle.com/api/v1/datasets/download/{dataset_name}"

try:
    print(f"Downloading from {url}...")
    response = requests.get(url, auth=auth, stream=True)
    
    if response.status_code == 200:
        zip_path = DATA_ROOT / "dataset.zip"
        
        # Download with progress
        total_size = int(response.headers.get('content-length', 0))
        downloaded = 0
        
        with open(zip_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    if total_size:
                        percent = (downloaded / total_size) * 100
                        print(f"Downloaded: {percent:.1f}%", end='\r')
        
        print(f"\n Downloaded to {zip_path}\n")
        
        # Extract
        print("Extracting files...")
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(DATA_ROOT)
        
        print(" Extraction complete!\n")
        
        # Clean up zip
        os.remove(zip_path)
        print(" Cleaned up zip file\n")
        
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"Error: {e}")

# Verify
print("Verifying dataset...")
def find_files(root, patterns):
    found = []
    for pat in patterns:
        found.extend(sorted(root.rglob(pat)))
    return found

jsonl_files = find_files(DATA_ROOT, ["*.jsonl", "*.json"])
parquet_files = find_files(DATA_ROOT, ["*.parquet"])

print(f" JSONL/JSON files: {len(jsonl_files)}")
print(f" Parquet files: {len(parquet_files)}")

if jsonl_files:
    print("\nDataset files found:")
    for f in jsonl_files[:10]:
        size_mb = f.stat().st_size / (1024*1024)
        print(f"  - {f.name} ({size_mb:.2f} MB)")

Downloaded: 100.0%
✓ Downloaded to /workspace/ecthr_data/dataset.zip

Extracting files...
✓ Extraction complete!

✓ Cleaned up zip file

Verifying dataset...
✓ JSONL/JSON files: 3
✓ Parquet files: 0

Dataset files found:
  - dev.jsonl (11.06 MB)
  - test.jsonl (11.97 MB)
  - train.jsonl (90.29 MB)


In [15]:
#  4. LOAD SPLITS 

def load_jsonl(path: Path):
    rows = []
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows

def load_split(keywords: List[str]):
    for kw in keywords:
        for f in jsonl_files:
            if kw in f.name.lower():
                print(f"   {kw}  JSONL: {f.name}  ({f.stat().st_size/1024:.0f} KB)")
                return load_jsonl(f)
        for f in parquet_files:
            if kw in f.name.lower():
                print(f"   {kw}  Parquet: {f.name}")
                return pd.read_parquet(f).to_dict("records")
    print(f"   No file matched for keywords: {keywords}")
    return []

print("Loading splits ")
raw_train = load_split(["train"])
raw_val   = load_split(["dev", "val", "valid", "validation"])
raw_test  = load_split(["test"])

print(f"train={len(raw_train):,}  val={len(raw_val):,}  test={len(raw_test):,}")
if raw_train:
    print(f"Sample keys: {list(raw_train[0].keys())}")


Loading splits …
  ✓ train → JSONL: train.jsonl  (92456 KB)
  ✓ dev → JSONL: dev.jsonl  (11324 KB)
  ✓ test → JSONL: test.jsonl  (12257 KB)
train=9,000  val=1,000  test=1,000
Sample keys: ['case_id', 'case_no', 'title', 'judgment_date', 'facts', 'applicants', 'defendants', 'allegedly_violated_articles', 'violated_articles', 'court_assessment_references', 'silver_rationales', 'gold_rationales']


In [ ]:
#  5. TOP-8 LABEL MAP 

def get_articles(rec) -> List[str]:
    for key in ("violated_articles", "labels", "violated_provisions",
                "gold_labels", "silver_rationales"):
        val = rec.get(key)
        if val is not None:
            if isinstance(val, list):
                return [str(v).strip() for v in val if v is not None and str(v).strip()]
            if isinstance(val, str) and val.strip():
                return [val.strip()]
    return []

def get_text(rec) -> str:
    for key in ("facts", "text", "case_text", "content"):
        val = rec.get(key)
        if val is not None:
            if isinstance(val, list):
                return " ".join(str(v) for v in val if v)
            return str(val)
    # Fallback: longest string values
    parts = [str(v) for v in rec.values() if isinstance(v, str) and len(v) > 30]
    return " ".join(parts)

# Count label frequencies
all_labels_raw = []
for rec in raw_train:
    all_labels_raw.extend(get_articles(rec))
label_counts = Counter(all_labels_raw)
if '13' in label_counts:
    del label_counts['13']
 

TOP_K      = 7

top_labels = [lbl for lbl, _ in label_counts.most_common(TOP_K)]
top_labels.sort()                # keep consistent alphabetical order
NUM_LABELS = len(top_labels)
label2idx  = {lbl: i for i, lbl in enumerate(top_labels)}
idx2label  = {i: lbl for lbl, i in label2idx.items()}

print(f"Top-{TOP_K} ECHR articles selected (Art 13 Excluded):")
print(f"{'Article':<14} {'Train count':>12} {'%':>6}")
print("-" * 36)
for lbl in top_labels:
    cnt = label_counts[lbl]
    print(f"  Art {lbl:<10s} {cnt:>12,} {100*cnt/len(raw_train):>6.1f}%")
print(f"label2idx = {label2idx}")

Top-7 ECHR articles selected (Art 13 Excluded):
Article         Train count      %
------------------------------------
  Art 10                  291    3.2%
  Art 2                   505    5.6%
  Art 3                 1,349   15.0%
  Art 5                 1,368   15.2%
  Art 6                 4,704   52.3%
  Art 8                   710    7.9%
  Art P1-1              1,421   15.8%
label2idx = {'10': 0, '2': 1, '3': 2, '5': 3, '6': 4, '8': 5, 'P1-1': 6}


## Section 2 — Tokenizer & Hierarchical Chunking

In [17]:
#  6. TOKENIZER 
# TOKENIZERS_PARALLELISM is already set to false before any imports,
# so num_workers > 0 in DataLoader will not cause the fork warning.

TOKENIZER_NAME = "bert-base-uncased"   # vocab = 30,522
CHUNK_SIZE     = 512                   # tokens per chunk
CHUNK_OVERLAP  = 64                    # overlapping tokens between chunks
MAX_CHUNKS     = 8                     # max chunks per document

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

print(f"Tokenizer : {TOKENIZER_NAME}")
print(f"Vocab     : {tokenizer.vocab_size:,}")
print(f"Chunk size: {CHUNK_SIZE} tokens  (overlap={CHUNK_OVERLAP})")
print(f"Max chunks: {MAX_CHUNKS}   max doc tokens seen = {MAX_CHUNKS * (CHUNK_SIZE - CHUNK_OVERLAP):,}")

# Quick sanity check
sample = "The applicant alleged a violation of Article 6 of the Convention."
enc    = tokenizer(sample, return_tensors="pt")
print(f"Sample : '{sample}'")
print(f"Tokens : {tokenizer.convert_ids_to_tokens(enc['input_ids'][0].tolist())}")


Tokenizer : bert-base-uncased
Vocab     : 30,522
Chunk size: 512 tokens  (overlap=64)
Max chunks: 8  → max doc tokens seen = 3,584
Sample : 'The applicant alleged a violation of Article 6 of the Convention.'
Tokens : ['[CLS]', 'the', 'applicant', 'alleged', 'a', 'violation', 'of', 'article', '6', 'of', 'the', 'convention', '.', '[SEP]']


In [18]:
#  7. HIERARCHICAL CHUNKING 
# Strategy: tokenise the full text, then slide a window of CHUNK_SIZE tokens
# with CHUNK_OVERLAP overlap. Each chunk is padded to CHUNK_SIZE.
# At inference, BDH encodes each chunk independently, then we mean-pool the
# chunk [CLS-equivalent] representations before the classification head.

def text_to_chunks(text: str,
                   tokenizer,
                   chunk_size: int = CHUNK_SIZE,
                   overlap: int    = CHUNK_OVERLAP,
                   max_chunks: int = MAX_CHUNKS):
    """
    Returns:
        input_ids      : (n_chunks, chunk_size)  LongTensor
        attention_masks: (n_chunks, chunk_size)  LongTensor
    """
    # Tokenize full text without truncation
    tokens = tokenizer(
        text,
        add_special_tokens=True,   # [CLS] ... [SEP]
        truncation=False,
        return_attention_mask=False,
    )["input_ids"]

    stride = chunk_size - overlap
    chunks_ids, chunks_mask = [], []

    start = 0
    while start < len(tokens) and len(chunks_ids) < max_chunks:
        end   = min(start + chunk_size, len(tokens))
        chunk = tokens[start:end]

        # Pad if shorter than chunk_size
        pad_len  = chunk_size - len(chunk)
        chunk    = chunk + [tokenizer.pad_token_id] * pad_len
        mask     = [1] * (len(tokens[start:end])) + [0] * pad_len

        chunks_ids.append(chunk)
        chunks_mask.append(mask)
        start += stride

    return (
        torch.tensor(chunks_ids,  dtype=torch.long),   # (n_chunks, chunk_size)
        torch.tensor(chunks_mask, dtype=torch.long),   # (n_chunks, chunk_size)
    )

#  Verify on a realistic document 
demo_text = " ".join(["The applicant alleged violations of the Convention."] * 200)
ids, masks = text_to_chunks(demo_text, tokenizer)
print(f"Demo document: {len(tokenizer(demo_text, truncation=False)['input_ids'])} tokens")
print(f"Chunks produced: {ids.shape[0]} × {ids.shape[1]} tokens each")
print(f"   {ids.shape[0] * ids.shape[1]} tokens total seen by model")


Token indices sequence length is longer than the specified maximum sequence length for this model (1602 > 512). Running this sequence through the model will result in indexing errors


Demo document: 1602 tokens
Chunks produced: 4 × 512 tokens each
  → 2048 tokens total seen by model


In [19]:
#  8. DATASET: PROCESS RECORDS (TOP-8 FILTER) 

def encode_labels(articles: List[str]) -> List[float]:
    vec = [0.0] * NUM_LABELS
    for art in articles:
        if art in label2idx:
            vec[label2idx[art]] = 1.0
    return vec

def process_records(records, split_name=""):
    texts, labels = [], []
    skip_no_top8, skip_empty = 0, 0
    for rec in records:
        t    = get_text(rec)
        arts = get_articles(rec)
        if not any(a in label2idx for a in arts):
            skip_no_top8 += 1
            continue
        if len(t.strip()) < 20:
            skip_empty += 1
            continue
        texts.append(t)
        labels.append(encode_labels(arts))
    if split_name:
        print(f"  {split_name}: {len(texts):,} kept | "
              f"skip_no_top8={skip_no_top8} | skip_empty={skip_empty}")
    return texts, labels

print("Processing splits ")
train_texts, train_labels = process_records(raw_train, "train")
val_texts,   val_labels   = process_records(raw_val,   "val")
test_texts,  test_labels  = process_records(raw_test,  "test")

#  Token-length stats (sample) 
sample_n   = min(300, len(train_texts))
sample_len = [
    len(tokenizer(t, truncation=False)["input_ids"])
    for t in train_texts[:sample_n]
]
print(f"Token lengths ({sample_n} train docs):")
print(f"  mean={np.mean(sample_len):.0f}  median={np.median(sample_len):.0f}"
      f"  p95={np.percentile(sample_len,95):.0f}  max={max(sample_len)}")
print(f"   docs needing >1 chunk: "
      f"{sum(l > CHUNK_SIZE for l in sample_len)} / {sample_n}")


Processing splits …
  train: 7,968 kept | skip_no_top8=1032 | skip_empty=0
  val: 813 kept | skip_no_top8=187 | skip_empty=0
  test: 822 kept | skip_no_top8=178 | skip_empty=0
Token lengths (300 train docs):
  mean=1776  median=1142  p95=4937  max=22637
  → docs needing >1 chunk: 211 / 300


In [20]:
#  9. CHUNKED DATASET 

class ECHRChunkedDataset(Dataset):
    """
    Pre-computes chunk tensors at construction time (once, on CPU).
    Each item: dict with
        chunk_ids   : (n_chunks, CHUNK_SIZE)
        chunk_masks : (n_chunks, CHUNK_SIZE)
        n_chunks    : int
        labels      : (NUM_LABELS,)
    """
    def __init__(self, texts, labels, tokenizer,
                 chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP, max_chunks=MAX_CHUNKS):
        self.labels     = labels
        self.chunk_size = chunk_size
        self.max_chunks = max_chunks

        print(f"  Pre-tokenizing {len(texts):,} documents ")
        self.chunks_ids  = []
        self.chunks_mask = []
        for text in texts:
            ids, msk = text_to_chunks(text, tokenizer, chunk_size, overlap, max_chunks)
            self.chunks_ids.append(ids)
            self.chunks_mask.append(msk)
        print(f"  Done.")

    def __len__(self): return len(self.labels)

    def __getitem__(self, idx):
        return {
            "chunk_ids"  : self.chunks_ids[idx],   # (n_chunks, chunk_size)
            "chunk_masks": self.chunks_mask[idx],   # (n_chunks, chunk_size)
            "n_chunks"   : self.chunks_ids[idx].shape[0],
            "labels"     : torch.tensor(self.labels[idx], dtype=torch.float),
        }

def collate_chunks(batch):
    """
    Pad batch to the same number of chunks (max in this batch).
    Output shapes:
        chunk_ids   : (B, max_n_chunks, CHUNK_SIZE)
        chunk_masks : (B, max_n_chunks, CHUNK_SIZE)
        doc_masks   : (B, max_n_chunks)   which chunk slots are real vs padding
        labels      : (B, NUM_LABELS)
    """
    max_nc = max(item["n_chunks"] for item in batch)
    B      = len(batch)
    C      = batch[0]["chunk_ids"].shape[1]

    chunk_ids_pad   = torch.zeros(B, max_nc, C, dtype=torch.long)
    chunk_masks_pad = torch.zeros(B, max_nc, C, dtype=torch.long)
    doc_masks       = torch.zeros(B, max_nc, dtype=torch.bool)

    for i, item in enumerate(batch):
        nc = item["n_chunks"]
        chunk_ids_pad[i,   :nc] = item["chunk_ids"]
        chunk_masks_pad[i, :nc] = item["chunk_masks"]
        doc_masks[i,       :nc] = True

    return {
        "chunk_ids"  : chunk_ids_pad,
        "chunk_masks": chunk_masks_pad,
        "doc_masks"  : doc_masks,
        "labels"     : torch.stack([item["labels"] for item in batch]),
    }

print("Building datasets ")
train_ds = ECHRChunkedDataset(train_texts, train_labels, tokenizer)
val_ds   = ECHRChunkedDataset(val_texts,   val_labels,   tokenizer)
test_ds  = ECHRChunkedDataset(test_texts,  test_labels,  tokenizer)

# Batch size: larger is fine now (each forward pass is per-chunk, not per-doc)
BATCH_SIZE = 4 if VRAM >= 14 else 8

# num_workers=0 avoids tokenizer fork issues (we pre-tokenized anyway)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=0, collate_fn=collate_chunks, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=0, collate_fn=collate_chunks, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=0, collate_fn=collate_chunks, pin_memory=True)

print(f"DataLoaders (batch={BATCH_SIZE}):")
print(f"  train={len(train_loader)} batches | val={len(val_loader)} | test={len(test_loader)}")


Building datasets …
  Pre-tokenizing 7,968 documents …
  Done.
  Pre-tokenizing 813 documents …
  Done.
  Pre-tokenizing 822 documents …
  Done.
DataLoaders (batch=4):
  train=1992 batches | val=204 | test=206


In [21]:
#  10. LABEL DISTRIBUTION 
label_freq_np = np.array(train_labels).sum(0)

fig, ax = plt.subplots(figsize=(10, 4))
bars = ax.bar(range(NUM_LABELS), label_freq_np,
              color=plt.cm.tab10(np.arange(NUM_LABELS)))
ax.set_xticks(range(NUM_LABELS))
ax.set_xticklabels([f"Art {top_labels[i]}" for i in range(NUM_LABELS)], fontsize=11)
ax.set_ylabel("Training examples")
ax.set_title(f"Top-{NUM_LABELS} ECHR articles  train set")
for i, v in enumerate(label_freq_np):
    ax.text(i, v + 30, f"{int(v)}", ha="center", fontsize=9)
plt.tight_layout()
plt.savefig("./label_dist.png", dpi=120, bbox_inches="tight")
plt.show()
print("Saved: ./label_dist.png")


Saved: ./label_dist.png


## Section 3 — BDH Architecture (Bidirectional, Fixed)




In [ ]:
#  11. BDH ARCHITECTURE 

@dataclass
class BDHConfig:
    n_layer  : int   = 6        # shared weight layers (Universal Transformer)
    n_embd   : int   = 256      # D: hidden dimension
    n_head   : int   = 4        # nh: attention heads
    mlp_internal_dim_multiplier: int = 64  # N = 64*256/4 = 4096
    dropout  : float = 0.15
    vocab_size : int = 30522    # bert-base-uncased
    num_labels : int = 8        # set at runtime

#  RoPE frequencies 
def get_freqs(n: int, theta: int, dtype) -> torch.Tensor:
    def quantize(t, q=2): return (t / q).floor() * q
    return (1.0 / (theta ** (quantize(torch.arange(0, n, 1, dtype=dtype)) / n))
            / (2 * math.pi))

#  BDH Attention  BIDIRECTIONAL
class BDHAttention(nn.Module):
    """
    BDH self-attention with RoPE.

    KEY FIX vs previous versions:
      The causal tril() mask is REMOVED.
      This is a classification encoder  all tokens must attend to all others.
      The tril mask was silently destroying contextual representations.

    NaN safety:
      - scores scaled by 1/sqrt(N) before softmax
      - all ops in fp32 regardless of AMP autocast
    """
    def __init__(self, config: BDHConfig):
        super().__init__()
        nh = config.n_head
        D  = config.n_embd
        N  = config.mlp_internal_dim_multiplier * D // nh
        self.N = N
        self.register_buffer(
            "freqs",
            get_freqs(N, theta=2**16, dtype=torch.float32).view(1, 1, 1, N),
        )

    @staticmethod
    def _rope(phases: torch.Tensor, v: torch.Tensor) -> torch.Tensor:
        v_rot = torch.stack((-v[..., 1::2], v[..., ::2]), dim=-1).view(*v.shape)
        ph = (phases % 1) * (2 * math.pi)
        return v * torch.cos(ph) + v_rot * torch.sin(ph)

    def forward(self, Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor) -> torch.Tensor:
        assert K is Q, "BDH requires K is Q (self-attention)"
        _, _, T, _ = Q.shape

        # Force fp32  prevents fp16 overflow on large N
        Q32 = Q.float()
        V32 = V.float()

        r_phases = (
            torch.arange(T, device=self.freqs.device, dtype=torch.float32)
            .view(1, 1, -1, 1)
        ) * self.freqs

        QR = self._rope(r_phases, Q32)              # (B, nh, T, N)

        #  BIDIRECTIONAL: no causal mask 
        scale  = 1.0 / math.sqrt(self.N)
        scores = (QR @ QR.mT) * scale               # (B, nh, T, T)
        # scores = scores.tril(diagonal=-1)          #  REMOVED (was the bug)

        out = scores @ V32                           # (B, nh, T, D)
        return out.to(Q.dtype)


#  BDH Backbone 
class BDHBackbone(nn.Module):
    """
    Shared-weight BDH encoder. Universal Transformer style:
    same encoder/encoder_v/decoder params reused across all L layers.

    Param budget (L=6, D=256, nh=4, N=4096):
        encoder   (nh, D, N): 4*256*4096 = 4,194,304
        encoder_v (nh, D, N): 4,194,304
        decoder (nh*N, D)   : 4,194,304
        embed   (vocab, D)  : 7,813,632
        Total backbone      : ~20.4M
    """
    def __init__(self, config: BDHConfig):
        super().__init__()
        self.config = config
        nh = config.n_head
        D  = config.n_embd
        N  = config.mlp_internal_dim_multiplier * D // nh

        # Shared weights  used identically at each of the L layers
        self.encoder   = nn.Parameter(torch.empty(nh, D, N).normal_(std=0.02))
        self.encoder_v = nn.Parameter(torch.empty(nh, D, N).normal_(std=0.02))
        self.decoder   = nn.Parameter(torch.empty(nh * N, D).normal_(std=0.02))

        self.attn  = BDHAttention(config)
        self.ln    = nn.LayerNorm(D, elementwise_affine=False, bias=False)
        self.embed = nn.Embedding(config.vocab_size, D)
        self.drop  = nn.Dropout(config.dropout)

        nn.init.normal_(self.embed.weight, std=0.02)

    def forward(self, idx: torch.Tensor) -> torch.Tensor:
        """
        Args: idx (B, T)
        Returns: hidden (B, T, D)
        """
        C  = self.config
        B, T = idx.shape
        nh = C.n_head
        N  = C.mlp_internal_dim_multiplier * C.n_embd // nh

        x = self.embed(idx).unsqueeze(1)    # (B, 1, T, D)
        x = self.ln(x)

        for _ in range(C.n_layer):
            # Project to sparse N-dim basis
            x_latent  = x @ self.encoder        # (B, nh, T, N)
            x_sparse  = F.relu(x_latent)        # sparse activations

            # BDH attention (bidirectional)
            yKV       = self.attn(Q=x_sparse, K=x_sparse, V=x)
            yKV       = self.ln(yKV)

            # Value encoder
            y_latent  = yKV @ self.encoder_v    # (B, nh, T, N)
            y_sparse  = F.relu(y_latent)

            # Hebbian product  pre × post synaptic correlation
            xy_sparse = x_sparse * y_sparse     # (B, nh, T, N)
            xy_sparse = self.drop(xy_sparse)

            # Decode back to D-dim
            yMLP = (
                xy_sparse.transpose(1, 2)            # (B, T, nh, N)
                .reshape(B, 1, T, nh * N)
                @ self.decoder                       # (B, 1, T, D)
            )
            y = self.ln(yMLP)
            x = self.ln(x + y)                   # residual

        return x.squeeze(1)                      # (B, T, D)


#  Hierarchical BDH Classifier 
class BDHClassifier(nn.Module):
    """
    Hierarchical BDH:
      1. Run BDHBackbone on each chunk independently  (B, n_chunks, T, D)
      2. Mean-pool each chunk's tokens  (B, n_chunks, D)   [chunk representations]
      3. Mean-pool valid chunks (using doc_mask)  (B, D)   [document representation]
      4. LN  Linear head  (B, num_labels)
    """
    def __init__(self, config: BDHConfig):
        super().__init__()
        self.backbone = BDHBackbone(config)
        self.ln_post  = nn.LayerNorm(config.n_embd, elementwise_affine=False, bias=False)
        self.head = nn.Sequential(
            nn.Linear(config.n_embd, config.n_embd),
            nn.GELU(),
            nn.Dropout(config.dropout),
            nn.Linear(config.n_embd, config.num_labels),
        )
        for m in self.head.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, std=0.02)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    @staticmethod
    def _token_pool(hidden: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        """Masked mean-pool over tokens. hidden: (B,T,D), mask: (B,T)  (B,D)"""
        m = mask.unsqueeze(-1).float()
        return (hidden * m).sum(1) / m.sum(1).clamp(min=1e-9)

    def forward(self,
                chunk_ids   : torch.Tensor,    # (B, n_chunks, T)
                chunk_masks : torch.Tensor,    # (B, n_chunks, T)
                doc_masks   : torch.Tensor,    # (B, n_chunks) bool
               ) -> torch.Tensor:              # (B, num_labels)
        B, n_chunks, T = chunk_ids.shape

        #  Encode all chunks 
        # Flatten to (B*n_chunks, T) for a single backbone pass
        ids_flat   = chunk_ids.view(B * n_chunks, T)
        masks_flat = chunk_masks.view(B * n_chunks, T)

        hidden_flat = self.backbone(ids_flat)                   # (B*n_chunks, T, D)

        # Mean-pool tokens within each chunk  chunk representation
        chunk_repr  = self._token_pool(hidden_flat, masks_flat) # (B*n_chunks, D)
        chunk_repr  = chunk_repr.view(B, n_chunks, -1)          # (B, n_chunks, D)

        #  Pool across valid chunks 
        dm = doc_masks.unsqueeze(-1).float()                    # (B, n_chunks, 1)
        doc_repr = (chunk_repr * dm).sum(1) / dm.sum(1).clamp(min=1e-9)  # (B, D)

        doc_repr = self.ln_post(doc_repr)
        return self.head(doc_repr)                              # (B, num_labels)

    def count_parameters(self):
        total = trainable = 0
        for p in self.parameters():
            total += p.numel()
            if p.requires_grad: trainable += p.numel()
        return total, trainable


In [23]:
#  12. INSTANTIATE MODEL 

CFG = BDHConfig(
    n_layer  = 6,
    n_embd   = 256,
    n_head   = 4,
    mlp_internal_dim_multiplier = 64,   # N = 64*256/4 = 4096
    dropout  = 0.15,
    vocab_size  = tokenizer.vocab_size,
    num_labels  = NUM_LABELS,
)

model = BDHClassifier(CFG).to(DEVICE)

#  Wrap with DataParallel if dual-GPU 
if USE_DP:
    model = nn.DataParallel(model)
    print(f" DataParallel enabled across {n_gpus} GPUs")

# Count params (unwrap DP for counting)
core_model = model.module if USE_DP else model
total_p, train_p = core_model.count_parameters()

nh = CFG.n_head; D = CFG.n_embd
N  = CFG.mlp_internal_dim_multiplier * D // nh

print("" * 62)
print("BDH CLASSIFIER  PARAMETER BUDGET")
print("" * 62)
print(f"  D (hidden)           : {D}")
print(f"  nh (heads)           : {nh}")
print(f"  N (sparse dim)       : {N}")
print(f"  Layers L (shared)    : {CFG.n_layer}")
print(f"  Dropout              : {CFG.dropout}")
print(f"  Vocab                : {CFG.vocab_size:,}")
print(f"  Labels               : {CFG.num_labels}")
print(f"  Chunk size           : {CHUNK_SIZE} × {MAX_CHUNKS} max chunks")
print("" * 62)
enc_p = nh * D * N
emb_p = CFG.vocab_size * D
hd_p  = D * D + D + D * CFG.num_labels
print(f"  encoder + encoder_v  : {enc_p*2:>12,}")
print(f"  decoder              : {enc_p:>12,}")
print(f"  embedding            : {emb_p:>12,}")
print(f"  head                 : {hd_p:>12,}")
print("" * 62)
print(f"  TOTAL                : {total_p:>12,}  ({total_p/1e6:.2f} M)")
print("" * 62)

#  Shape / NaN check 
dummy_ids   = torch.zeros(2, 3, 16, dtype=torch.long,  device=DEVICE)  # (B,n_chunks,T)
dummy_masks = torch.ones( 2, 3, 16, dtype=torch.long,  device=DEVICE)
dummy_dm    = torch.ones( 2, 3,     dtype=torch.bool,  device=DEVICE)

with torch.no_grad():
    out = model(dummy_ids, dummy_masks, dummy_dm)

assert tuple(out.shape) == (2, NUM_LABELS), f"Unexpected shape {out.shape}"
assert not torch.isnan(out).any(), "NaN in forward pass!"
print(f"  Forward pass   {tuple(out.shape)}  ")
print(f"  NaN check      clean  ")
print("" * 62)


══════════════════════════════════════════════════════════════
BDH CLASSIFIER — PARAMETER BUDGET
══════════════════════════════════════════════════════════════
  D (hidden)           : 256
  nh (heads)           : 4
  N (sparse dim)       : 4096
  Layers L (shared)    : 6
  Dropout              : 0.15
  Vocab                : 30,522
  Labels               : 7
  Chunk size           : 512 × 8 max chunks
──────────────────────────────────────────────────────────────
  encoder + encoder_v  :    8,388,608
  decoder              :    4,194,304
  embedding            :    7,813,632
  head                 :       67,584
──────────────────────────────────────────────────────────────
  TOTAL                :   20,464,135  (20.46 M)
══════════════════════════════════════════════════════════════
  Forward pass  → (2, 7)  ✓
  NaN check     → clean  ✓
══════════════════════════════════════════════════════════════


## Section 4 — Training Setup

In [24]:
#  13. LOSS, OPTIMIZER, SCHEDULER 

#  pos_weight (restored)  corrects for class imbalance 
label_freq_t = torch.tensor(label_freq_np, dtype=torch.float)
pos_weight   = ((len(train_labels) - label_freq_t) / label_freq_t.clamp(min=1))
pos_weight   = pos_weight.clamp(max=20.0).to(DEVICE)
criterion    = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction="mean")

print("pos_weight per label:")
for i, lbl in enumerate(top_labels):
    print(f"  Art {lbl:<8s}  freq={int(label_freq_np[i]):>5,}  pw={pos_weight[i].item():.3f}")

#  Hyper-parameters 
EPOCHS       = 8
LR           = 2e-4
WEIGHT_DECAY = 0.05
WARMUP_FRAC  = 0.08
GRAD_CLIP    = 1.0
PATIENCE     = 10       # early-stopping patience (val loss)

TOTAL_STEPS  = EPOCHS * len(train_loader)
WARMUP_STEPS = int(WARMUP_FRAC * TOTAL_STEPS)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LR, betas=(0.9, 0.95), weight_decay=WEIGHT_DECAY, eps=1e-8,
)

def lr_lambda(step):
    if step < WARMUP_STEPS: return step / max(1, WARMUP_STEPS)
    progress = (step - WARMUP_STEPS) / max(1, TOTAL_STEPS - WARMUP_STEPS)
    return 0.5 * (1.0 + math.cos(math.pi * progress))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

USE_AMP = DEVICE.type == "cuda"
scaler  = GradScaler("cuda", enabled=USE_AMP)

print(f"Training config:")
print(f"  Epochs={EPOCHS}  LR={LR}  WD={WEIGHT_DECAY}")
print(f"  Warmup={WARMUP_STEPS}  TotalSteps={TOTAL_STEPS:,}")
print(f"  GradClip={GRAD_CLIP}  Patience={PATIENCE}")
print(f"  AMP={USE_AMP}  Batch={BATCH_SIZE}  GPUs={n_gpus}")


pos_weight per label:
  Art 10        freq=  291  pw=20.000
  Art 2         freq=  505  pw=14.778
  Art 3         freq=1,349  pw=4.907
  Art 5         freq=1,368  pw=4.825
  Art 6         freq=4,704  pw=0.694
  Art 8         freq=  710  pw=10.223
  Art P1-1      freq=1,421  pw=4.607
Training config:
  Epochs=8  LR=0.0002  WD=0.05
  Warmup=1274  TotalSteps=15,936
  GradClip=1.0  Patience=10
  AMP=True  Batch=4  GPUs=1


In [25]:
#  14. EVALUATION UTILITIES 

def compute_metrics(all_logits: np.ndarray, all_labels: np.ndarray):
    """Hard threshold at logit=0 (sigmoid=0.5)."""
    preds = (all_logits > 0).astype(int)
    return {
        "micro_f1"    : f1_score(all_labels, preds, average="micro",   zero_division=0),
        "macro_f1"    : f1_score(all_labels, preds, average="macro",   zero_division=0),
        "sample_f1"   : f1_score(all_labels, preds, average="samples", zero_division=0),
        "micro_prec"  : precision_score(all_labels, preds, average="micro", zero_division=0),
        "micro_rec"   : recall_score(all_labels,    preds, average="micro", zero_division=0),
        "per_label_f1": f1_score(all_labels, preds, average=None, zero_division=0),
    }

def find_best_thresholds(all_logits: np.ndarray, all_labels: np.ndarray):
    """
    Per-label threshold tuning on validation set.
    Sweeps sigmoid probabilities 0.1..0.9 and picks the one maximising F1
    for each label independently.
    Returns: array of optimal thresholds in logit space.
    """
    probs = 1.0 / (1.0 + np.exp(-all_logits))   # sigmoid
    best_thresh = np.zeros(all_labels.shape[1])

    for c in range(all_labels.shape[1]):
        best_f1, best_t = 0.0, 0.5
        for t in np.linspace(0.1, 0.9, 17):
            preds_c = (probs[:, c] > t).astype(int)
            f1_c    = f1_score(all_labels[:, c], preds_c, zero_division=0)
            if f1_c > best_f1:
                best_f1, best_t = f1_c, t
        best_thresh[c] = best_t

    # Convert to logit space for consistency with inference
    eps = 1e-7
    best_thresh = np.clip(best_thresh, eps, 1 - eps)
    return np.log(best_thresh / (1.0 - best_thresh))   # logit thresholds

@torch.no_grad()
def evaluate(loader, thresholds=None):
    """
    Args:
        thresholds: per-label logit thresholds (array of shape [NUM_LABELS]).
                    If None, uses 0.0 (= sigmoid 0.5) for all labels.
    """
    core = model.module if USE_DP else model
    core.eval()
    all_logits, all_labels_list, total_loss = [], [], 0.0

    for batch in loader:
        ids   = batch["chunk_ids"].to(DEVICE)
        masks = batch["chunk_masks"].to(DEVICE)
        dm    = batch["doc_masks"].to(DEVICE)
        lbls  = batch["labels"].to(DEVICE)

        with autocast("cuda", enabled=USE_AMP):
            logits = model(ids, masks, dm) if USE_DP else core(ids, masks, dm)
            loss   = criterion(logits, lbls)

        total_loss += loss.item()
        all_logits.append(logits.cpu().float().numpy())
        all_labels_list.append(lbls.cpu().numpy())

    all_logits_np = np.concatenate(all_logits, axis=0)
    all_labels_np = np.concatenate(all_labels_list, axis=0)

    if thresholds is not None:
        # Apply per-label thresholds
        preds = (all_logits_np > thresholds[None, :]).astype(int)
        m = {
            "micro_f1"    : f1_score(all_labels_np, preds, average="micro",   zero_division=0),
            "macro_f1"    : f1_score(all_labels_np, preds, average="macro",   zero_division=0),
            "sample_f1"   : f1_score(all_labels_np, preds, average="samples", zero_division=0),
            "micro_prec"  : precision_score(all_labels_np, preds, average="micro", zero_division=0),
            "micro_rec"   : recall_score(all_labels_np,    preds, average="micro", zero_division=0),
            "per_label_f1": f1_score(all_labels_np, preds, average=None, zero_division=0),
        }
    else:
        m = compute_metrics(all_logits_np, all_labels_np)

    m["loss"]       = total_loss / len(loader)
    m["raw_logits"] = all_logits_np    # stored for threshold search
    m["raw_labels"] = all_labels_np
    return m

print(" Evaluation utilities ready")


✓ Evaluation utilities ready


## Section 5 — Training (Early Stopping, Per-Label Threshold)

In [26]:
#  15. TRAINING LOOP 

CKPT_DIR = Path("./bdh_checkpoints")
CKPT_DIR.mkdir(exist_ok=True)

history = {k: [] for k in
           ["train_loss","val_loss","val_micro_f1","val_macro_f1","val_sample_f1","lr"]}

best_val_f1      = -1.0
best_val_loss    = float("inf")
patience_counter = 0
best_thresholds  = np.zeros(NUM_LABELS)  # will be updated from val logits
t0_total         = time.time()

for epoch in range(1, EPOCHS + 1):
    #  Training 
    model.train()
    epoch_loss, epoch_steps, nan_count = 0.0, 0, 0
    t0 = time.time()

    for batch_idx, batch in enumerate(train_loader):
        ids   = batch["chunk_ids"].to(DEVICE, non_blocking=True)
        masks = batch["chunk_masks"].to(DEVICE, non_blocking=True)
        dm    = batch["doc_masks"].to(DEVICE, non_blocking=True)
        lbls  = batch["labels"].to(DEVICE, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with autocast("cuda", enabled=USE_AMP):
            logits = model(ids, masks, dm)
            loss   = criterion(logits, lbls)

        if torch.isnan(loss):
            nan_count += 1
            print(f"   NaN at ep{epoch} b{batch_idx}  skipping")
            optimizer.zero_grad(set_to_none=True)
            continue

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        epoch_loss  += loss.item()
        epoch_steps += 1

        log_every = max(1, len(train_loader) // 4)
        if (batch_idx + 1) % log_every == 0:
            cur_lr = optimizer.param_groups[0]["lr"]
            print(f"  Ep{epoch:02d} [{batch_idx+1:>4d}/{len(train_loader)}] "
                  f"loss={loss.item():.4f}  lr={cur_lr:.2e}")

    avg_loss = epoch_loss / max(1, epoch_steps)

    #  Validation (default threshold) 
    val_m = evaluate(val_loader, thresholds=None)

    #  Per-label threshold search on val set 
    epoch_thresholds = find_best_thresholds(val_m["raw_logits"], val_m["raw_labels"])
    val_m_tuned      = evaluate(val_loader, thresholds=epoch_thresholds)

    cur_lr = optimizer.param_groups[0]["lr"]
    history["train_loss"].append(avg_loss)
    history["val_loss"].append(val_m["loss"])
    history["val_micro_f1"].append(val_m_tuned["micro_f1"])
    history["val_macro_f1"].append(val_m_tuned["macro_f1"])
    history["val_sample_f1"].append(val_m_tuned["sample_f1"])
    history["lr"].append(cur_lr)

    elapsed = time.time() - t0
    nan_str  = f"   {nan_count} NaN batches" if nan_count else ""

    print(f"{'='*65}")
    print(f"  Epoch {epoch:02d}/{EPOCHS}  [{elapsed:.0f}s]{nan_str}")
    print(f"  train_loss={avg_loss:.4f}  val_loss={val_m['loss']:.4f}")
    print(f"  [default thr]  micro_F1={val_m['micro_f1']:.4f}  "
          f"macro_F1={val_m['macro_f1']:.4f}")
    print(f"  [tuned  thr]   micro_F1={val_m_tuned['micro_f1']:.4f}  "
          f"macro_F1={val_m_tuned['macro_f1']:.4f}  "
          f"sample_F1={val_m_tuned['sample_f1']:.4f}")
    print(f"  P={val_m_tuned['micro_prec']:.4f}  "
          f"R={val_m_tuned['micro_rec']:.4f}  LR={cur_lr:.2e}")

    #  Save checkpoint every epoch 
    epoch_ckpt_path = CKPT_DIR / f"epoch_{epoch:02d}.pt"
    torch.save({
        "epoch"        : epoch,
        "model_state"  : (model.module if USE_DP else model).state_dict(),
        "optim_state"  : optimizer.state_dict(),
        "sched_state"  : scheduler.state_dict(),
        "config"       : CFG,
        "label2idx"    : label2idx,
        "idx2label"    : idx2label,
        "top_labels"   : top_labels,
        "thresholds"   : epoch_thresholds,
        "val_metrics"  : val_m_tuned,
        "history"      : history,
    }, epoch_ckpt_path)
    print(f"   Epoch checkpoint saved  {epoch_ckpt_path.name}")

    #  Checkpoint on best tuned micro-F1 
    if val_m_tuned["micro_f1"] > best_val_f1:
        best_val_f1     = val_m_tuned["micro_f1"]
        best_thresholds = epoch_thresholds
        torch.save({
            "epoch"        : epoch,
            "model_state"  : (model.module if USE_DP else model).state_dict(),
            "optim_state"  : optimizer.state_dict(),
            "sched_state"  : scheduler.state_dict(),
            "config"       : CFG,
            "label2idx"    : label2idx,
            "idx2label"    : idx2label,
            "top_labels"   : top_labels,
            "thresholds"   : best_thresholds,
            "val_metrics"  : val_m_tuned,
            "history"      : history,
        }, CKPT_DIR / "best_model.pt")
        print(f"   Best checkpoint saved  (micro_F1={best_val_f1:.4f})")
        patience_counter = 0
    else:
        patience_counter += 1
        print(f"  No improvement  ({patience_counter}/{PATIENCE})")
        if patience_counter >= PATIENCE:
            print(f" Early stopping at epoch {epoch}")
            break

    print("=" * 65)

print(f"Training done in {(time.time()-t0_total)/60:.1f} min")
print(f"Best val micro-F1 (tuned threshold) = {best_val_f1:.4f}")
print(f"Per-label optimal thresholds (logit): {best_thresholds.round(3)}")


  Ep01 [ 498/1992] loss=0.8829  lr=7.82e-05
  Ep01 [ 996/1992] loss=2.5645  lr=1.56e-04
  Ep01 [1494/1992] loss=1.1191  lr=2.00e-04
  Ep01 [1992/1992] loss=0.3737  lr=1.99e-04
  Epoch 01/8  [676s]
  train_loss=0.8099  val_loss=0.6211
  [default thr]  micro_F1=0.6916  macro_F1=0.6474
  [tuned  thr]   micro_F1=0.7355  macro_F1=0.7103  sample_F1=0.7424
  P=0.7106  R=0.7622  LR=1.99e-04
  💾 Epoch checkpoint saved → epoch_01.pt
  ✓ Best checkpoint saved  (micro_F1=0.7355)
  Ep02 [ 498/1992] loss=0.2201  lr=1.97e-04
  Ep02 [ 996/1992] loss=0.0917  lr=1.93e-04
  Ep02 [1494/1992] loss=1.0664  lr=1.89e-04
  Ep02 [1992/1992] loss=0.1159  lr=1.84e-04
  Epoch 02/8  [682s]
  train_loss=0.5805  val_loss=0.6176
  [default thr]  micro_F1=0.7581  macro_F1=0.7373
  [tuned  thr]   micro_F1=0.7843  macro_F1=0.7693  sample_F1=0.7934
  P=0.7403  R=0.8338  LR=1.84e-04
  💾 Epoch checkpoint saved → epoch_02.pt
  ✓ Best checkpoint saved  (micro_F1=0.7843)
  Ep03 [ 498/1992] loss=0.2153  lr=1.77e-04
  Ep03 [ 996

In [ ]:
import json
from collections import defaultdict

@torch.no_grad()
def extract_activations_for_dataset(model, dataloader, device, idx2label, top_n=50):
    """
    Run inference on all test cases and extract:
    1. Active neuron IDs + activation values per case
    2. Build a global neuronconcept co-activation index
    """
    model.eval()
    core = model.module if hasattr(model, 'module') else model
    backbone = core.backbone
    C = backbone.config
    nh, N = C.n_head, C.mlp_internal_dim_multiplier * C.n_embd // C.n_head
    
    records = []
    coactivation = defaultdict(lambda: defaultdict(int))
    concept_occurrence = defaultdict(int)
    
    for batch_idx, (chunk_ids, chunk_masks, doc_masks, labels) in enumerate(dataloader):
        chunk_ids   = chunk_ids.to(device)
        chunk_masks = chunk_masks.to(device)
        doc_masks   = doc_masks.to(device)
        
        logits = core(chunk_ids, chunk_masks, doc_masks)
        probs  = torch.sigmoid(logits).cpu().numpy()
        
        # Get stored sparse activations from last layer
        xy = backbone._last_xy_sparse  # (B_flat, nh, T, N)
        B = chunk_ids.shape[0]
        n_chunks = chunk_ids.shape[1]
        
        for b in range(B):
            # Pool this sample's activations
            start = b * n_chunks
            end   = start + n_chunks
            sample_xy = xy[start:end].mean(dim=(0, 2)).reshape(-1)  # (16384,)
            
            active_mask = sample_xy > 0
            active_idx  = torch.where(active_mask)[0].cpu().tolist()
            active_vals = sample_xy[active_mask].cpu().tolist()
            sorted_pairs = sorted(zip(active_idx, active_vals), key=lambda x: -x[1])
            
            # Get predictions
            sample_probs = probs[b]
            pred_labels = [idx2label[i] for i in range(len(sample_probs)) 
                          if sample_probs[i] >= thresholds.get(i, 0.5)]
            true_labels = [idx2label[i] for i in range(labels.shape[1]) 
                          if labels[b, i] > 0.5]
            
            # === THIS IS WHERE YOU ADD YOUR CONCEPT LABELING ===
            # Option A: Use Claude API to label concepts (as you did)
            # Option B: Use the concepts_from_claude.txt mapping
            # For now, store raw neuron data; concepts added post-hoc
            
            records.append({
                "case_id": f"batch-{batch_idx}-{b}",
                "violated_articles": true_labels,
                "predicted_labels": pred_labels,
                "active_neurons": [p[0] for p in sorted_pairs[:top_n]],
                "neuron_act_values": [round(p[1], 6) for p in sorted_pairs[:top_n]],
                "sparsity": round(len(active_idx) / 16384, 10),
            })
            
            # Build co-activation index:
            # For each concept associated with this case's predicted articles,
            # record which neurons were active
            for nid in active_idx:
                for art in pred_labels:
                    coactivation[f"art_{art}_neuron"][str(nid)] = \
                        coactivation[f"art_{art}_neuron"].get(str(nid), 0) + 1
    
    return records, dict(coactivation)

# Run extraction
print("Extracting activations from test set...")
records, coact = extract_activations_for_dataset(
    model, test_loader, DEVICE, idx2label
)

# Save
with open("example_records.json", "w") as f:
    json.dump(records, f, indent=2)
print(f"Saved {len(records)} records to example_records.json")

# If you already have coactivation_raw.json and concept_occurrence.json
# from your concept mapping notebook, those are used directly by serve.py.
# Otherwise, save the per-article neuron index:
with open("coactivation_raw.json", "w") as f:
    json.dump(coact, f, indent=2)
print("Saved coactivation_raw.json")

## Section 6 — Results & Diagnostics

In [27]:
#  16. TRAINING CURVES 
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

ax = axes[0]
ax.plot(history["train_loss"], "o-", label="train", ms=6)
ax.plot(history["val_loss"],   "s-", label="val",   ms=6)
ax.set_title("Loss"); ax.set_xlabel("Epoch"); ax.legend(); ax.grid(alpha=0.3)

# Overfitting gap annotation
if len(history["train_loss"]) > 1:
    gap = history["train_loss"][-1] - history["val_loss"][-1]
    ax.annotate(f"gap={gap:+.3f}", xy=(len(history["train_loss"])-1,
                history["train_loss"][-1]), fontsize=9, color="red")

ax = axes[1]
ax.plot(history["val_micro_f1"],  "o-", label="micro-F1",  ms=6)
ax.plot(history["val_macro_f1"],  "s-", label="macro-F1",  ms=6)
ax.plot(history["val_sample_f1"], "^-", label="sample-F1", ms=6)
ax.set_title("Validation F1 (tuned threshold)")
ax.set_xlabel("Epoch"); ax.legend(); ax.grid(alpha=0.3)

ax = axes[2]
ax.plot(history["lr"], ".-", color="green")
ax.set_title("LR Schedule"); ax.set_xlabel("Step"); ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig("./training_curves.png", dpi=120, bbox_inches="tight")
plt.show()
print("Saved: ./training_curves.png")


Saved: ./training_curves.png


In [28]:
#  17. PER-LABEL F1 (best checkpoint, tuned thresholds) 

# Load best checkpoint
ckpt = torch.load(CKPT_DIR / "best_model.pt", map_location=DEVICE, weights_only=False)
core_model = model.module if USE_DP else model
core_model.load_state_dict(ckpt["model_state"])
best_thresholds = ckpt["thresholds"]

val_m = evaluate(val_loader, thresholds=best_thresholds)
plf1  = val_m["per_label_f1"]

#  Bar chart 
colors = ["#d73027" if f < 0.2 else "#fc8d59" if f < 0.4
          else "#fee090" if f < 0.6 else "#91bfdb" if f < 0.8
          else "#4575b4" for f in plf1]

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(range(NUM_LABELS), plf1, color=colors)
ax.set_xticks(range(NUM_LABELS))
ax.set_xticklabels([f"Art {top_labels[i]}" for i in range(NUM_LABELS)], fontsize=11)
ax.axhline(val_m["micro_f1"], color="black", ls="--", lw=1.5,
           label=f"micro-F1={val_m['micro_f1']:.3f}")
ax.axhline(val_m["macro_f1"], color="gray",  ls=":",  lw=1.5,
           label=f"macro-F1={val_m['macro_f1']:.3f}")
for i, v in enumerate(plf1):
    ax.text(i, v + 0.02, f"{v:.2f}", ha="center", fontsize=10)
ax.set_ylim(0, 1.1)
ax.set_ylabel("F1 Score")
ax.set_title(f"Per-Article F1  Best Checkpoint (Tuned Thresholds)")
ax.legend(); ax.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.savefig("./per_label_f1.png", dpi=120, bbox_inches="tight")
plt.show()

#  Table 
print(f"{'Article':<12}  {'F1':>6}  {'Precision':>10}  {'Recall':>8}  "
      f"{'Threshold':>10}  {'Support':>8}")
print("-" * 62)

preds_tuned = (val_m["raw_logits"] > best_thresholds[None, :]).astype(int)
for i, lbl in enumerate(top_labels):
    p_i   = precision_score(val_m["raw_labels"][:, i], preds_tuned[:, i], zero_division=0)
    r_i   = recall_score(val_m["raw_labels"][:, i],    preds_tuned[:, i], zero_division=0)
    sup_i = int(label_freq_np[i])
    thr_i = 1.0 / (1.0 + np.exp(-best_thresholds[i]))   # to sigmoid probability
    print(f"  Art {lbl:<8s}  {plf1[i]:.4f}  {p_i:>10.4f}  {r_i:>8.4f}  "
          f"{thr_i:>10.3f}  {sup_i:>8,}")

print(f"Micro-F1  : {val_m['micro_f1']:.4f}")
print(f"  Macro-F1  : {val_m['macro_f1']:.4f}")
print(f"  Sample-F1 : {val_m['sample_f1']:.4f}")
print(f"  Precision : {val_m['micro_prec']:.4f}")
print(f"  Recall    : {val_m['micro_rec']:.4f}")


Article           F1   Precision    Recall   Threshold   Support
--------------------------------------------------------------
  Art 10        0.8049      0.8250    0.7857       0.850       291
  Art 2         0.8174      0.8103    0.8246       0.900       505
  Art 3         0.8337      0.8000    0.8705       0.700     1,349
  Art 5         0.7903      0.7946    0.7861       0.450     1,368
  Art 6         0.7906      0.7493    0.8367       0.350     4,704
  Art 8         0.6120      0.5833    0.6437       0.500       710
  Art P1-1      0.8472      0.8188    0.8777       0.900     1,421
Micro-F1  : 0.7931
  Macro-F1  : 0.7852
  Sample-F1 : 0.8007
  Precision : 0.7679
  Recall    : 0.8199


In [29]:
#  18. TEST SET EVALUATION 
print("Evaluating on test set with best thresholds ")
test_m = evaluate(test_loader, thresholds=best_thresholds)

print(f"Test set results (best checkpoint + tuned thresholds):")
print(f"  Loss      = {test_m['loss']:.4f}")
print(f"  Micro-F1  = {test_m['micro_f1']:.4f}")
print(f"  Macro-F1  = {test_m['macro_f1']:.4f}")
print(f"  Sample-F1 = {test_m['sample_f1']:.4f}")
print(f"  Precision = {test_m['micro_prec']:.4f}")
print(f"  Recall    = {test_m['micro_rec']:.4f}")

print(f"Per-label test F1:")
test_plf1 = test_m["per_label_f1"]
for i, lbl in enumerate(top_labels):
    print(f"  Art {lbl:<8s}  {test_plf1[i]:.4f}")


Evaluating on test set with best thresholds …
Test set results (best checkpoint + tuned thresholds):
  Loss      = 1.2586
  Micro-F1  = 0.7716
  Macro-F1  = 0.7675
  Sample-F1 = 0.7783
  Precision = 0.7643
  Recall    = 0.7791
Per-label test F1:
  Art 10        0.6479
  Art 2         0.8850
  Art 3         0.8359
  Art 5         0.7645
  Art 6         0.7709
  Art 8         0.6894
  Art P1-1      0.7787


In [32]:
#  20. SAVE ALL OUTPUTS 
import shutil

outputs = [
    "./bdh_checkpoints/best_model.pt",
    "./training_curves.png",
    "./per_label_f1.png",
    "./label_dist.png",
]

print("Output files:")
for p in outputs:
    path = Path(p)
    if path.exists():
        print(f"   {path.name}  [{path.stat().st_size/1e6:.2f} MB]")
    else:
        print(f"   {path.name}  (not found)")

print(" All outputs are in /kaggle/working/  commit the notebook to download them.")


Output files:
  ✓ best_model.pt  [245.66 MB]
  ✓ training_curves.png  [0.10 MB]
  ✓ per_label_f1.png  [0.04 MB]
  ✓ label_dist.png  [0.03 MB]
✓ All outputs are in /kaggle/working/ — commit the notebook to download them.
